# IDS703 Final Project

In [1]:
import numpy as np
import torch
import operator
from operator import itemgetter
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from nltk.corpus import brown
import warnings
import random

nltk.download('brown')
from nltk.corpus import treebank
nltk.download('treebank')
from nltk.corpus import conll2000
nltk.download('conll2000')
nltk.download('universal_tagset')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

## Step 2: Generative Probability Base Model
1. Model explanation
2. Accuracy explanation

Sklearn classification metrics report
sklearn library F1, Recall, Precision
Sklearn ROC curves

qualitatively: edge cases
quantitatively: 

In [2]:
#import pre-marked Part of speech tagging text data from the nltk library
treebank = treebank.tagged_sents(tagset='universal')
brown = brown.tagged_sents(tagset='universal')
conll = conll2000.tagged_sents(tagset='universal')

In [3]:
# Generative Probabilistic Model: Hidden Markov Model

def transition(data):
  """
  Generate a transition matrix from corpus
  Sample output: [[0, 0.1, ...], 
                  [0.2, 0, ...], 
                  [0, 0]...]]
  Each row represents the current tag, 
  each column represents a tag that followed the current tag
  """
  transition_matrix = []
  tags = {}
  next_tags = {}

  for s in data:
    # modify sentence by add a start tag and a end tag (<start>, <end>)
    sentence = s.copy()
    #sentence.insert(0, ('', '<start>'))
    #sentence.append(('', '<end>'))
    for i in range(len(sentence) - 1):
      tag = sentence[i][1]
      next_tag = sentence[i + 1][1]

      row_index = None # transition matrix index of current tag
      column_index = None # transition matrix index of next tag
      # check if current tag exist in the tags dictionary
      if tag in tags.keys():
        # check next tag exist in the next_tags dictionary
        row_index = tags[tag]
      else:
        row_index = len(tags.keys())
        tags[tag] = row_index
        if len(transition_matrix) == 0:
          transition_matrix.append([])
        else:
          new_row = [0]*len(transition_matrix[0])
          transition_matrix.append(new_row)

      # check if word recorded
      if next_tag in next_tags.keys():
        column_index = next_tags[next_tag]
        transition_matrix[row_index][column_index] += 1
      else:
        column_index = len(next_tags.keys())
        next_tags[next_tag] = column_index
        for row in transition_matrix:
          row.append(0)
        transition_matrix[row_index][column_index] = 1

  transition_matrix = np.array(transition_matrix) + 1 # added smoothing
  transition_matrix = transition_matrix/transition_matrix.sum(axis = 1, keepdims = True)
  return transition_matrix

def emission(data):
  """
  Generate a observation likelihood matrix (aka emission matrix) from corpus
  Sample output: [[0, 0.1, ...], 
                  [0.2, 0, ...], 
                  [0, 0]...]]
  Each row represents the current tag, 
  Each column represents a word that's associated with the current tag
  """
  # create a n x m matrix, n = number of unique words, m = number of unique tags
  emission_matrix = []
  word_index = {} # record word to column index
  tag_index = {} # record tag to row index

  for sentence in data:
    for pair in sentence:
      word = pair[0]
      tag = pair[1]
      row_index = None # tag index in matrix
      column_index = None # word index in matrix

      # check if tag recorded
      row_index = None
      column_index = None

      # check if tag recorded
      if tag in tag_index.keys():
        row_index = tag_index[tag]
      # tag not exist, add new tag row
      else:
        row_index = len(tag_index.keys())
        tag_index[tag] = row_index
        if len(emission_matrix) == 0:
          emission_matrix.append([])
        else:
          new_row = [0]*len(emission_matrix[0])
          emission_matrix.append(new_row)
      
      # check if word recorded
      if word in word_index.keys():
        column_index = word_index[word]
        emission_matrix[row_index][column_index] += 1
      else:
        column_index = len(word_index.keys())
        word_index[word] = column_index
        for row in emission_matrix:
          row.append(0)
        emission_matrix[row_index][column_index] = 1
  emission_matrix = np.array(emission_matrix) + 1 # added smoothing
  emission_matrix = emission_matrix/emission_matrix.sum(axis = 1, keepdims = True)
  return emission_matrix, word_index, tag_index

def initial_state(data, tag_index):
  initial_state_dist = [0]*len(tag_index.keys())
  tags = []
  counter = 0
  for sentence in data:
    counter += 1
    tag = sentence[0][1]
    index = tag_index[tag]
    initial_state_dist[index] += 1
  
  initial_state_dist = np.array(initial_state_dist) + 1
  initial_state_dist = initial_state_dist/initial_state_dist.sum()
  return initial_state_dist

#OOV
def OOV(t1, t3, data, emission_matrix, tag_index):
  # compute OOV matrix
  tags_pairs = {}
  
  # populate matrix:
  # (tag1, tage2, tag3) ==> {(tag1, tag3): {verb: 1, noun: 3, ...}}
  for sentence in data:
    for i in range(len(sentence) - 2):
      tag1 = sentence[i][1]
      tag2 = sentence[i + 1][1]
      tag3 = sentence[i + 2][1]
      pair = (tag1, tag3)
      if pair in tags_pairs.keys():
        if tag2 in tags_pairs[pair].keys():
          tags_pairs[pair][tag2] += 1
        else:
          tags_pairs[pair][tag2] = 1
      else:
          tags_pairs[pair] = {}
          tags_pairs[pair][tag2] = 1

  # compute distribution
  for key in list(tags_pairs.keys()):
    pair_list = tags_pairs[key]
    total = sum(pair_list.values())
    for key2 in list(pair_list.keys()):
      tags_pairs[key][key2] = tags_pairs[key][key2]/total

  # get appropriate substitute
  pair = (t1, t3)
  tags = tags_pairs[pair]
  max_tag = max(zip(tags.values(), tags.keys()))[1]
  max_tag_index = tag_index[max_tag]
  max_tag_words = emission_matrix[max_tag_index]
  max_word_index = np.argmax(max_tag_words)
  #return max_word_index
  return max_word_index

# part of speech
def POS(data):
  """
  This function generates the curcial components of a part of speech tagging HMM
  1. transition matrix
  2. emission matrix/observation likelihood matrix
  3. initial state distribution 
  """

  # transition matrix: record the probability of a tag followed by a tag
  # procedure: get a tag, count the differenct tags that comes after
  transition_df = transition(data)

  # observation likelihood matrix: record the probability of a tag associated to a word
  emission_matrix, word_index, tag_index = emission(data)

  # initial state distribution
  initial_state_distribution = initial_state(data, tag_index)

  return transition_df, emission_matrix, initial_state_distribution, word_index, tag_index

In [4]:
# Viterbi Algorithm (Decoder of the Hidden Markov Model)
def viterbi(obs, pi, A, B):
  '''
  @source: https://en.wikipedia.org/wiki/Viterbi_algorithm pseudocode from Wikipedia
  '''
  # obs - the observations, list of ints
  # pi - the initial state probabilities, list of floats
  # A - the state transition probability matrix [2D numpy array]
  # B - the observation probability aka emission matrix [2D numpy array]
  # states - list of ints
  
  states = np.zeros(len(obs))
  pi = np.log(pi + 1)
  A = np.log(A+1)
  B = np.log(B+1)
  T1 = np.zeros((A.shape[0], len(obs)))
  T2 = np.zeros((A.shape[0], len(obs)))
  T1[:, 0] = pi * B[:, obs[0]]
  T2[:, 0] = 0
  for j in range(1, len(obs)):
    for k in range(0, A.shape[0]):
      T1[k,j] = np.max([T1[i, j - 1] * A[i, k] * B[k, obs[j]] for i in range(0, A.shape[0])])
      T2[k,j] = np.argmax([T1[i, j - 1] * A[i, k] * B[k, obs[j]] for i in range(0, A.shape[0])])
  states[-1] = np.argmax(T1[:, len(obs) - 1])
  for m in range(-1, -(len(obs)), -1):
    states[m - 1] = T2[int(states[m]), m]
  return states

def words_to_index(sentence, word_index, tag_index, emission_matrix, corpus):
  obs = []
  for i in range(len(sentence)):
    pair = sentence[i]
    word = pair[0]
    if word in word_index.keys():
      index = word_index[word]
      obs.append(index)
    else:
      #OOV
      pair1 = sentence[i - 1][1]
      pair2 = sentence[i][1]
      pair3 = sentence[i + 1][1]
      index = OOV(pair1, pair3, corpus, emission_matrix, tag_index)
      obs.append(index)
  return obs 

## Step 3: Neural Network Model

See the other Jupyter Notebook for Neural Network Code.

## Step 4: Text Generator Using Viterbi

In [5]:
from nltk.corpus import brown
import pickle
nltk.download('brown')
data = brown.tagged_sents(tagset='universal')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [6]:
# Generative Probabilistic Model: Hidden Markov Model
def transition_gen(data):
  """
  Generate a transition matrix from corpus
  Sample output: [[0, 0.1, ...], 
                  [0.2, 0, ...], 
                  [0, 0]...]]
  Each row represents the current tag, 
  each column represents a tag that followed the current tag
  """
  transition_matrix = []
  tags = {}
  next_tags = {}

  for s in data:
    # modify sentence by add a start tag and a end tag (<start>, <end>)
    sentence = s.copy()
    #sentence.insert(0, ('', '<start>'))
    #sentence.append(('', '<end>'))
    for i in range(len(sentence) - 1):
      tag = sentence[i][1]
      next_tag = sentence[i + 1][1]

      row_index = None # transition matrix index of current tag
      column_index = None # transition matrix index of next tag
      # check if current tag exist in the tags dictionary
      if tag in tags.keys():
        # check next tag exist in the next_tags dictionary
        row_index = tags[tag]
      else:
        row_index = len(tags.keys())
        tags[tag] = row_index
        if len(transition_matrix) == 0:
          transition_matrix.append([])
        else:
          new_row = [0]*len(transition_matrix[0])
          transition_matrix.append(new_row)

      # check if word recorded
      if next_tag in next_tags.keys():
        column_index = next_tags[next_tag]
        transition_matrix[row_index][column_index] += 1
      else:
        column_index = len(next_tags.keys())
        next_tags[next_tag] = column_index
        for row in transition_matrix:
          row.append(0)
        transition_matrix[row_index][column_index] = 1

  transition_matrix = np.array(transition_matrix) + 0.1 # added smoothing
  transition_matrix = transition_matrix/transition_matrix.sum(axis = 1, keepdims = True)
  return transition_matrix, tags, next_tags

def emission_gen(data):
  """
  Generate a observation likelihood matrix (aka emission matrix) from corpus
  Sample output: [[0, 0.1, ...], 
                  [0.2, 0, ...], 
                  [0, 0]...]]
  Each row represents the current tag, 
  Each column represents a word that's associated with the current tag
  """
  # create a n x m matrix, n = number of unique words, m = number of unique tags
  emission_matrix = []
  word_index = {} # record word to column index
  tag_index = {} # record tag to row index

  for sentence in data:
    for pair in sentence:
      word = pair[0]
      tag = pair[1]
      row_index = None # tag index in matrix
      column_index = None # word index in matrix

      # check if tag recorded
      row_index = None
      column_index = None

      # check if tag recorded
      if tag in tag_index.keys():
        row_index = tag_index[tag]
      # tag not exist, add new tag row
      else:
        row_index = len(tag_index.keys())
        tag_index[tag] = row_index
        if len(emission_matrix) == 0:
          emission_matrix.append([])
        else:
          new_row = [0]*len(emission_matrix[0])
          emission_matrix.append(new_row)
      
      # check if word recorded
      if word in word_index.keys():
        column_index = word_index[word]
        emission_matrix[row_index][column_index] += 1
      else:
        column_index = len(word_index.keys())
        word_index[word] = column_index
        for row in emission_matrix:
          row.append(0)
        emission_matrix[row_index][column_index] = 1
  emission_matrix = np.array(emission_matrix) + 0.1 # added smoothing
  emission_matrix = emission_matrix/emission_matrix.sum(axis = 1, keepdims = True)
  return emission_matrix, word_index, tag_index

def initial_state_gen(data, tag_index):
  initial_state_dist = [0]*len(tag_index.keys())
  tags = []
  counter = 0
  for sentence in data:
    counter += 1
    tag = sentence[0][1]
    index = tag_index[tag]
    initial_state_dist[index] += 1
  
  initial_state_dist = np.array(initial_state_dist) + 0.1 # added smoothing
  initial_state_dist = initial_state_dist/initial_state_dist.sum()
  return initial_state_dist

In [7]:
def tags_generator(prob_df, pi, tag_index, num = 10):
  """
  prob: transition matrix
  num: number of tokens to generate
  """
  tokens = []
  p = prob_df.to_numpy()
  results = [pi]
  for i in range(num-1):
    results.append(pi@p)
    p = p@prob_df.to_numpy()
    
  for r in results:
    tokens.append(np.random.choice(tag_index, p = r))
  return tokens

def words_generator(prob_df, tag_lst):

  words = []
  word_bank = list(prob_df.columns)
  for tag in tag_lst:
    words_dist = prob_df.loc[tag].values
    word = np.random.choice(word_bank, p = words_dist)
    words.append(word)
  return words


def create_synthetic_data(tag_sequences, word_sequences):
  data = []
  for i in range(len(tag_sequences)):
    sentence = []
    for j in range(len(tag_sequences[i])):
      token = (word_sequences[i][j], tag_sequences[i][j])
      sentence.append(token)
    data.append(sentence)
  return data

In [8]:
# extract transition matrix from Viterbi Algorithm 
transition_matrix, tags, next_tags = transition_gen(data) # generate transition matrix
sorted_tags = sorted(tags.items(), key=operator.itemgetter(1))
sorted_tags = [x[0] for x in sorted_tags]
sorted_next_tags = sorted(next_tags.items(), key=operator.itemgetter(1))
sorted_next_tags = [x[0] for x in sorted_next_tags]

transition_df = pd.DataFrame(transition_matrix)
transition_df.columns = list(sorted_next_tags)
transition_df.index = list(sorted_tags)

In [9]:
emission_matrix, word_index, tag_index = emission_gen(data) # likelihood of word associating with a tag 
sorted_tag_index = sorted(tag_index.items(), key=operator.itemgetter(1))
sorted_tag_index = [x[0] for x in sorted_tag_index]

emission_df = pd.DataFrame(emission_matrix)
emission_df.columns = list(word_index.keys())
emission_df.index = list(sorted_tag_index)

In [10]:
pi = initial_state_gen(data, tag_index)

In [11]:
# generate synthetic data in 2D array
# generate tags
tag_sequences = []
for i in range(100):
  sequence = tags_generator(transition_df, pi, sorted_tag_index)
  if sequence[0] == ".":
    continue
  else:
    tag_sequences.append(sequence)

# generate words
word_sequences = []
for tags in tag_sequences:
  words = words_generator(emission_df, tags)
  word_sequences.append(words)

# combine tags and words into appropriate input format
data = create_synthetic_data(tag_sequences, word_sequences)

In [12]:
# save generate synthetic data into pickle file for potential future use
with open('test.txt', 'rb') as f:
    syn_data = pickle.load(f)

In [13]:
from nltk.corpus import brown
nltk.download('brown')
from nltk.corpus import treebank
nltk.download('treebank')

treebank = treebank.tagged_sents(tagset='universal')
brown = brown.tagged_sents(tagset='universal')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


In [14]:
# test step 2
# train model on syn_data
A, B, pi, word_index, tag_index = POS(brown)

# test model on syn_data
states_data = []
for test in syn_data:
  obs = words_to_index(test, word_index, tag_index, B, brown)
  states = viterbi(obs, pi, A, B)
  states_data.append(states)

In [15]:
# Evaluation Metrics of POS tagging
def accuracy(states, tag_index, test):
  test_states = []
  tag_states = []
  for state in states:
    for index in state:
      key = [k for k, v in tag_index.items() if v == index][0]
      tag_states.append(key)
  for seq in test:
    for t in seq:
      key = t[1]
      test_states.append(key)

  print(classification_report(test_states, tag_states, digits=3))

In [16]:
accuracy(states_data, tag_index, syn_data)

              precision    recall  f1-score   support

           .      0.940     0.988     0.963        80
         ADJ      0.792     0.805     0.798       118
         ADP      0.763     0.879     0.817        66
         ADV      0.850     0.654     0.739        52
        CONJ      0.912     0.775     0.838        40
         DET      0.867     0.964     0.913       358
        NOUN      0.813     0.763     0.787       114
         NUM      0.917     0.550     0.687        20
        PRON      0.800     0.900     0.847        40
         PRT      0.667     0.421     0.516        19
        VERB      0.844     0.529     0.651        51
           X      0.000     0.000     0.000         2

    accuracy                          0.845       960
   macro avg      0.764     0.686     0.713       960
weighted avg      0.842     0.845     0.838       960



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Step 5: Testing With Real Data

In [17]:
test_index = np.random.randint(0, len(brown), 500)
train_index = list(set(list(range(0, len(brown)))) - set(test_index))

In [18]:
train_data = itemgetter(*train_index)(brown)
test_data = itemgetter(*test_index)(brown)

In [19]:
# test step 2
# train model on syn_data
A, B, pi, word_index, tag_index = POS(train_data)

# test model on syn_data
states_data = []
for test in test_data:
  obs = words_to_index(test, word_index, tag_index, B, train_data)
  states = viterbi(obs, pi, A, B)
  states_data.append(states)

In [20]:
# Evaluation Metrics of POS tagging
def accuracy(states, tag_index, test):
  test_states = []
  tag_states = []
  for state in states:
    for index in state:
      key = [k for k, v in tag_index.items() if v == index][0]
      tag_states.append(key)
  for seq in test:
    for t in seq:
      tag = t[1]
      test_states.append(tag)
  print(classification_report(test_states, tag_states, digits=3))

In [21]:
accuracy(states_data, tag_index, test_data)

              precision    recall  f1-score   support

           .      0.928     1.000     0.963      1250
         ADJ      0.791     0.773     0.782       768
         ADP      0.785     0.968     0.867      1258
         ADV      0.882     0.826     0.853       535
        CONJ      0.987     1.000     0.993       296
         DET      0.751     0.976     0.849      1186
        NOUN      0.961     0.774     0.857      2424
         NUM      0.950     0.800     0.869       120
        PRON      0.901     0.958     0.928       400
         PRT      0.781     0.433     0.557       247
        VERB      0.933     0.862     0.896      1549
           X      0.917     0.611     0.733        18

    accuracy                          0.872     10051
   macro avg      0.881     0.832     0.846     10051
weighted avg      0.882     0.872     0.870     10051

